# Método de Arnoldi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

**Subespacio de Krylov.** Para una matriz $A\in ℝ^{n\times n}$  y un vector $v\inℝ^{n}$ se define el subespacio de Krylov $\mathcal{K}_{m}(A,v)$ de la siguiente manera
$$
\mathcal{K}_{m}(A,v) = ⟨v, Av, A^{2}v, \ldots, A^{m-1}v ⟩
$$

La dimensión de este espacio dependerá del grado del vector $v$ respecto a la matriz $A$, donde el grado se entiende como el grado del polinomio $p(x)$ tal que $p(A)v = 0$. Para un $m$ mayor o igual al grado $\mu$  del vector $v$ el subespacio de Krylov $\mathcal{K}_m$ es exactamente igual a $\mathcal{K}_{\mu}$. Nótese que todo vector perteneciente a este subespacio puede ser visto como $q(A)v$ donde $q(x)$ es un polinomio.


Recordemos que la idea de construir estos espacios es usar métodos de proyección (ortogonales u oblicuos) para resolver un sistema de la forma $Ax =b$ donde la matriz $A$ y el vector $b$ son conocidos. Por la estructura del subespacio de Krylov se quiere hacer una aproximación de $A^{-1}$ por un polinomio $r(x)$ evaluado en la matriz $A$, así, $x = A^{-1}b \approx r(A) b$.



En los métodos de proyección se resuelve un sistema $Ax=b$ 'grande' mediante la solución de sistemas lineales 'pequeños', estos sistemas lineales pequeños se pueden resolver fácilmente si se conoce una base ortogonal de los espacios de proyección (generalmente representados por $\mathcal{K}$ y $\mathcal{L}$). El método de Arnoldi mostrado en este cuaderno busca una base ortonormal del subespacio de Krylov $\mathcal{K}_{m}(A,v)$.

En primer lugar, es importante observar porqué no se usa un método de ortonormalización tradicional. Se definen la matriz $A_0$ y el vector $v$ para conformar el subespacio de Krylov $\mathcal{K}_{m}(A_0,v)$.

In [ ]:
# Inicialización de la matriz A0
n = 100 #Tamaño de la matriz
A0 = 2*np.random.random((n,n))-1 #Matriz aleatoria entre -1 y 1
v0 = 2*np.random.random((n,))-1

Construimos la matriz que tiene como columnas el conjunto generador del subespacio de Krylov.

In [ ]:
def Krylov(A,v,m):
  Kr = np.zeros((A.shape[0],m))
  Kr[:,0] = v.reshape((v.shape[0],)) / np.linalg.norm(v)
  for a in range(1,m):
    Kr[:,a] = A @ Kr[:,a-1]
  return(Kr)
print("El subespacio de Krylov con un conjunto generador de tamaño 4 es: \n")
print(Krylov(A0,v0,4))

El subespacio de Krylov con un conjunto generador de tamaño 4 es: 

[[ 1.11750143e-01  3.07368079e-01  2.75166863e+00 -1.83400856e+00]
 [-2.21174192e-02 -1.20775203e+00  8.81369487e-01 -1.60644107e+01]
 [-1.23596784e-01  1.07904025e+00  5.21006370e+00 -2.79492288e+00]
 [ 1.07670451e-01  3.98904422e-01 -2.75065953e+00  1.89949396e+01]
 [ 3.68488848e-02 -2.38255790e-01 -1.32289527e+00 -6.98961444e+00]
 [ 8.45468345e-02 -6.31272719e-01  1.20263377e-01  6.10748150e+00]
 [ 1.59635857e-01  2.04922767e-01  1.35716714e+00  1.71812865e+00]
 [-1.18891781e-01  1.69509852e-01  1.88956154e+00  2.34810985e+01]
 [-3.84926910e-03 -3.82925087e-01 -3.46547661e+00 -9.99343324e+00]
 [-1.22297438e-01  2.58095186e-01 -1.66113408e+00 -2.87648889e+01]
 [ 1.06621954e-01  6.51145604e-01  5.59668498e+00  3.31994313e-01]
 [-1.17105119e-01  1.80210934e-01 -9.56132539e-01  3.33094861e+01]
 [ 6.22900281e-02  8.81849221e-01  1.61279078e+00  2.40168075e+01]
 [ 1.34701366e-01 -1.19221588e+00 -4.21577261e+00  2.18152516

Observemos cómo cambia el número de condición de la matriz que contiene el conjunto generador del subespacio de Krylov a medida que aumenta $m$, o sea, cuando aumenta el tamaño del conjunto generador.

In [ ]:
NumCond = []
m_max = 20
for i in range(m_max):
  NumCond.append(np.linalg.cond(Krylov(A0,v0,i+1)))
print(tabulate(np.transpose([range(1,m_max+1),NumCond]), headers = ["m","Número de condición"]))

  m    Número de condición
---  ---------------------
  1            1
  2            5.74838
  3           35.3741
  4          189.702
  5         1020.21
  6         5449.23
  7        26675.2
  8       136914
  9       742310
 10            3.81315e+06
 11            2.16947e+07
 12            1.31832e+08
 13            7.21275e+08
 14            4.01502e+09
 15            2.26938e+10
 16            1.19768e+11
 17            6.58987e+11
 18            3.74708e+12
 19            2.39151e+13
 20            1.41148e+14


Nótese que tomando $m=10$ el número de condición supera $10^{6}$ indicando un problema de casi dependencia lineal entre los vectores del conjunto generador del subespacio de Krylov.

Ahora, apliquemos el método de Gram-Schdmit para ortonormalizar la base del subespacio de Krylov.

In [ ]:
def QRCGS(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v = v - R[i, j] * Q[:, i]

        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]

    return(Q, R)

Q, R = QRCGS(Krylov(A0,v0,m=100))
print("Matriz Q (ortogonal):")
print(Q)
print("\nMatriz R (triangular superior):")
print(R)

Matriz Q (ortogonal):
[[ 0.11175014  0.03999544  0.056602   ...  0.08397394  0.08248253
  -0.08318919]
 [-0.02211742 -0.21084919  0.08083735 ... -0.05621841 -0.05575656
   0.05602143]
 [-0.12359678  0.20683825  0.12218089 ... -0.23927763 -0.23910756
   0.2388915 ]
 ...
 [-0.16036644  0.01551954  0.15965078 ...  0.16863809  0.16949224
  -0.16878896]
 [-0.12364637  0.08899327 -0.07510782 ... -0.04437506 -0.04442957
   0.0445689 ]
 [ 0.12259089  0.00699364 -0.08603219 ...  0.04558869  0.04452148
  -0.04520335]]

Matriz R (triangular superior):
[[ 1.00000000e+00  7.27747542e-01  4.70412204e+00 ... -4.01783420e+73
   6.45250617e+73  9.47516529e+74]
 [ 0.00000000e+00  5.65169897e+00  8.18627081e+00 ...  6.26876192e+73
  -3.20318387e+74  2.20435137e+74]
 [ 0.00000000e+00  0.00000000e+00  3.35424310e+01 ... -5.64180998e+73
   4.62574343e+74 -1.57729726e+75]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.05517474e+68
  -1.48414098e+67  2.91623034e+68]
 [ 0.00000000e+00  0.0000000

In [ ]:
ErrorOrtogonalizacion = []
ErrorFactorizacion = []
Rango = [50,100,150,200,300]
for i in Rango:
  A = Krylov(A0,v0,m=i)
  Q, R = QRCGS(A)
  ErrorOrtogonalizacion.append(np.linalg.norm(Q.T @ Q - np.eye(i)))
  ErrorFactorizacion.append(np.linalg.norm(A - Q @ R)/np.linalg.norm(A))
print(tabulate(np.transpose((Rango,ErrorOrtogonalizacion,ErrorFactorizacion)), headers=["m","Error en ortogonalizacion","Error en factorizacion"]))

  m    Error en ortogonalizacion    Error en factorizacion
---  ---------------------------  ------------------------
 50                  6.95111e-10               3.05012e-16
100                 28.2253                    4.10854e-16
150                 76.9707                    5.87712e-16
200                126.299                     5.7019e-16
300                134.584                   nan


El error de ortogonalización va aumentando a medida de que el tamaño del conjunto generador aumenta.

En la siguiente sección se soluciona este error usando el método de Arnoldi.

## Algoritmo con Gram Schmidt clásico

**Algoritmo básico: Arnoldi**
Para solucionar el anterior problema se emplea el método de Arnoldi. El pseudocódigo usanfo el método de Gram-Schdmit clásico se muestra a continuación.

Dada una matriz $A\in ℝ^{n\times n}$ se inicializa la matriz $H \in ℝ^{m+1 \times m}$ y $V \in ℝ^{n \times m+1}$.
1. Escoja un vector $v_1$ unitario
2. Para $j=1,2,\ldots, m$:
3. > Calcule $h_{ij} = (Av_{i},v_{j})$ para $i=1,\ldots,j$
4. > Calcule $w_{j} = Av_{j} - \sum^{j}_{i=1} h_{ij}v_{j}$
5. > $h_{j+1,j} = ||w_{j}||_{2}$
6. > Si $h_{j+1,j}=0$ entonces: Parar
7. > $v_{j+1} = w_{j}/h_{j+1,j}$
8. Fin









In [ ]:
def ArnoldiCGS(A,v,m):
    # A es la matriz del subespacio de Krylov
    # v es el vector del subespacio de Krylov
    # m es el número de iteraciones o también el tamaño del subespacio de Krylov

    # Inicializamos las variables necesarias
    n = A.shape[0]
    H = np.zeros((m + 1, m))
    V = np.zeros((n, m + 1))

    # El primer elemento de la base será el vector v normalizado
    V[:, 0] = v / np.linalg.norm(v)

    for j in range(m):
        #
        w = A @ V[:, j]
        for i in range(j + 1):
            #Producto interno entre w y cada vector de la base
            H[i, j] = np.dot(w, V[:, i])

        # Al vector w se le restan las componentes correspondientes a los
        # primeros i<=j vectores de la base en una sola operación
        w = w - V @ H[:,j]

        if j < m - 1:
            H[j + 1, j] = np.linalg.norm(w)
            if H[j + 1, j] == 0:
              # Si la norma del vector w es cero entonces
              # el subepsacio de Krylov tiene dimensión j
                break
            # Definimos el siguiente vector de la base.
            # Este vector es ortogonal a todos los anteriores vectores de la base
            # guardados en la matriz V. Además, el nuevo vector es unitario.
            V[:, j + 1] = w / H[j + 1, j]

    #Esta función retorna la matriz H y la matriz V.
    #La matriz H es una matriz Hessengberg relacionada con A.
    #La matriz V contiene como columnas la base del subespacio
    #de Krylov asociados con la matriz A y el vector v normalizado.

    return(H[:m, :m], V[:, :m])


In [ ]:
# Obtener la descomposición de Arnoldi
H, V = ArnoldiCGS(A0,v0, m=4)

#Información de la descomposición
print("Matriz H:\n", H)
#print("\nMatriz V:\n", V)
print("\nNuúmero de condición de la matriz V:\n", np.linalg.cond(V))
print("\nOrtogonalidad: \n", V.T@ V)

Matriz H:
 [[ 0.72774754  0.73862843 -0.56387365  0.19510563]
 [ 5.65169897  0.72071439 -0.08659384 -0.93367339]
 [ 0.          5.9349288  -0.40577793  0.18866759]
 [ 0.          0.          5.36894755  0.38593469]]

Nuúmero de condición de la matriz V:
 1.0000000000000004

Ortogonalidad: 
 [[ 1.00000000e+00 -5.68578529e-17 -2.07659152e-17  1.36626318e-17]
 [-5.68578529e-17  1.00000000e+00 -1.46643332e-17  1.67512505e-18]
 [-2.07659152e-17 -1.46643332e-17  1.00000000e+00 -7.55985816e-19]
 [ 1.36626318e-17  1.67512505e-18 -7.55985816e-19  1.00000000e+00]]


Nótese la significativa mejora que hay en el número de condición de la matriz luego de ortogonalizar por medio del algoritmo de Arnoldi. Un número de condición cercano a $1$ indica que las columnas de la matriz $V$ son casi ortogonales como se ve en la salida del código anterior.

Recordemos que el método de Arnoldi puede ser descrito también como la siguiente iteración
$$
A V_m = V_m H_m + h_{m+1,m}v_{m+1} e_{m}^{T}
$$
donde $V_m$ es la base ortogonal del $m$-ésimo subespacio de Krylov. De la anterior ecuación se deduce que
$$
V^T_m A V_m = H_m
$$
donde $H_m$ es la matriz que tiene todos los productos internos de la forma $v^{T}_{j} A v_{i}$; esta es una matriz de Hessenberg. Comprobemos este último hecho.

In [ ]:
print(np.linalg.norm( V.T @ A0 @ V -H))

6.111227779539192e-15


A continuación observamos cómo cambia el número de condición de la matriz $V_m$ y la norma de $V^T_m A V_m - H_m$ a medida que aumenta $m$.

In [ ]:
Teorema = []
NumCondi = []
Rango = [5,10,20,50,100,200,500,1000]
for m in Rango:
  H, V = ArnoldiCGS(A0,v0,m)
  Teorema.append(np.linalg.norm( V.T @ A0 @ V -H))
  NumCondi.append(np.linalg.cond(V))
print(tabulate(np.transpose((Rango,NumCondi,Teorema)), headers=["m","Condición de la base","Cumplimiento del teorema"]))

   m    Condición de la base    Cumplimiento del teorema
----  ----------------------  --------------------------
   5                  1                      6.37947e-15
  10                  1                      7.27495e-15
  20                  1                      1.0437e-14
  50                  1                      2.16498e-14
 100                  1                      2.38538e-12
 200                 10.0499               395.206
 500                 20.025               3124.89
1000                 30.0167             10524.3


## Algoritmo con Gram Schmidt modificado

Una pequeña pero significativa modificación es realizar el método de Arnoldi con el algoritmo modificado de Gram-Schmidt.

In [ ]:
def ArnoldiMGS(A,v,m):
    #El objetivo de esta función es calcular una base ortonormal del
    #subespacio de Kyrlov asociados a la matriz A y al vector v de tamaño m
    #usando una Gram-Schmidt modificado

    # A es la matriz del subespacio de Krylov
    # v es el vector del subespacio de Krylov
    # m es el número de iteraciones o también el tamaño del subespacio de Krylov

    # Inicializamos las variables necesarias
    n = A.shape[0]
    H = np.zeros((m + 1, m))
    V = np.zeros((n, m + 1)) #En esta matriz se guardan los vectores de la base

    # El primer elemento de la base será el vector v normalizado
    V[:, 0] = v / np.linalg.norm(v)

    for j in range(m):
        w = A @ V[:, j]
        for i in range(j + 1):
            #Se calcula el producto punto entre cada vector de la base
            #y el nuevo vector w
            H[i, j] = np.dot(w, V[:, i])
            #Al vector w se le restan las componentes correspondientes a los
            # primeros i<=j vectores de la base en una cada iteración
            w = w - H[i, j] * V[:, i]

        if j < m - 1:
            H[j + 1, j] = np.linalg.norm(w)
            if H[j + 1, j] == 0:
                break
            V[:, j + 1] = w / H[j + 1, j]

    return(H[:m, :m], V[:, :m])


In [ ]:
# Obtener la descomposición de Arnoldi
H, V = ArnoldiMGS(A0,v0, m=4)

#Información de la descomposición
print("Matriz H:\n", H)
#print("\nMatriz V:\n", V)
print("\nNuúmero de condición de la matriz V:\n", np.linalg.cond(V))
print("\nOrtogonalidad: \n", V.T@ V)

Matriz H:
 [[ 0.72774754  0.73862843 -0.56387365  0.19510563]
 [ 5.65169897  0.72071439 -0.08659384 -0.93367339]
 [ 0.          5.9349288  -0.40577793  0.18866759]
 [ 0.          0.          5.36894755  0.38593469]]

Nuúmero de condición de la matriz V:
 1.0000000000000004

Ortogonalidad: 
 [[ 1.00000000e+00 -5.68578529e-17 -2.60035187e-17  4.13179083e-17]
 [-5.68578529e-17  1.00000000e+00  4.51468127e-18 -2.40545585e-17]
 [-2.60035187e-17  4.51468127e-18  1.00000000e+00  5.20360669e-17]
 [ 4.13179083e-17 -2.40545585e-17  5.20360669e-17  1.00000000e+00]]


Nótese que en este caso cuando se tiene un $m$ mayor a $100$ el método sigue comportandose bien en términos del número de condición de la matriz $V_m$ y no crece tanto la norma de la matriz $V^T_m A V_m - H_m$.

In [ ]:
Teorema = []
NumCondi = []
Rango = [5,10,20,50,100,200,500,1000]
for m in Rango:
  H, V = ArnoldiMGS(A0,v0,m)
  Teorema.append(np.linalg.norm( V.T @ A0 @ V -H))
  NumCondi.append(np.linalg.cond(V))
print(tabulate(np.transpose((Rango,NumCondi,Teorema)), headers=["m","Condición de la base","Cumplimiento del teorema"]))

   m    Condición de la base    Cumplimiento del teorema
----  ----------------------  --------------------------
   5                       1                 5.0103e-15
  10                       1                 6.92156e-15
  20                       1                 1.11919e-14
  50                       1                 2.24759e-14
 100                       1                 1.27684e-12
 200                       1                82.0042
 500                       1               259.32
1000                       1               550.101


In [2]:
def house(x):
  n = x.shape[0]
  s = x[1:n].T @ x[1:n]
  v = np.zeros((n,1))
  v[0] = 1
  v[1:n] = x[1:n]
  if s == 0:
        beta = 0
  else:
      mu = np.sqrt(x[0]**2 + s)
      if x[0] <= 0 :
          v[0] = x[0] - mu
      else:
          v[0] = -s /(x[0]+mu)
      beta = 2*((v[0])**2)/(s + (v[0])**2)
      v = v/v[0]
  refl = x - beta * (x.T @ v) *v
  P = np.eye(n) - beta * (v @ v.T)
  return(v,beta,refl,P)

In [3]:
def ArnoldiHouseholder(A,v,m):


    n = A.shape[0]
    H = np.zeros((m + 1, m))
    V = np.zeros((n, m + 1))


    V[:, 0] = v / np.linalg.norm(v)
    for j in range(m):
        w = np.zeros((n,1))
        ho = house(V[j:, j].reshape((n-j,1)))
        w[j:] = ho[0]
        H[j:,j-1] = ho[2]
        V[:,j] =  ho[3] @ np.eye(n)[:,j]
        if j<m:
          V[:,j+1]= ho[3] @ A @ V[:,j]


    return(V)

In [4]:
A = np.array([[4, -1, 0, -2],
                  [-1, 4, -1, 0],
                  [0, -1, 4, -1],
                  [0, 0, -1, 4]])

ArnoldiHouseholder(A,np.ones(4),1)

ValueError: ignored